In [ ]:
import random
from random import choice
import numpy as np
import time
import torch as T
import torch
import torch.nn.functional as F
device = T.device("cpu") 

In [ ]:
EMBEDDING_SIZE = 150
SHAPE_EMBEDDING_SIZE = 6

USE_SHAPE_ENCODING = True
USE_TYPE_ENCODING = True
USE_VALUE_ENCODING = True

EMBEDDING_DATA_NAME = 'Single_100000_synthetic'

In [ ]:
api2indx = torch.load(EMBEDDING_DATA_NAME + "_api2indx.pt")

In [ ]:
def encode_values_to_code(tensor):
    tensor = tensor.clone()
    tensor[(tensor>=100) & (tensor<1000)] = 100
    tensor[(tensor>=1000)] = 101
    tensor[(tensor<=-20) & (tensor>-100)] = -20
    tensor[(tensor<=-100) & (tensor>-1000)] = -21
    tensor[(tensor<=-1000)] = -22
    return tensor

In [ ]:
import sklearn.datasets
import torch
import numpy as np
import torch.nn.functional as F

def tensor_flatten_pad(tensor, embed_size=EMBEDDING_SIZE, shape_embed_size=SHAPE_EMBEDDING_SIZE):
    if not isinstance(tensor, torch.Tensor):
        tensor = torch.tensor(tensor)
    
    t_flatten = torch.flatten(tensor)

    if USE_VALUE_ENCODING:
        t_flatten = encode_values_to_code(t_flatten)

    padding_length = embed_size - list(t_flatten.shape)[-1]
    p1d = (0,padding_length)
    t_pad = F.pad(input=t_flatten, pad=p1d, mode='constant', value=0)
    
    if USE_TYPE_ENCODING:
        type_padding = 0
        if tensor.dtype == torch.bool:
            type_padding = 1
    
    if USE_SHAPE_ENCODING:
        t_shape = list(tensor.shape)
        padding_length = shape_embed_size -1 - len(t_shape)
        p1d = (0,padding_length)
        s_pad = F.pad(input=torch.tensor(t_shape), pad=p1d, mode='constant', value=0)

        t_pad_list = t_pad.tolist()
        s_pad_list = s_pad.tolist()
        if USE_TYPE_ENCODING:
            tensor_embedding = torch.tensor([type_padding] + [-1] + t_pad_list + [-1] + s_pad_list + [-1])
        else:
            tensor_embedding = torch.tensor(t_pad_list + [-1] + s_pad_list + [-1])
    
    else:
        t_pad_list = t_pad.tolist()
        if USE_TYPE_ENCODING:
            tensor_embedding = torch.tensor([type_padding] + [-1] + t_pad_list + [-1])
        else:
            tensor_embedding = torch.tensor(t_pad_list + [-1])

    return(tensor_embedding.float())

In [ ]:
class Net(torch.nn.Module):
  def __init__(self, len_api):
    super(Net, self).__init__()

    first_layer_size = EMBEDDING_SIZE
    if USE_SHAPE_ENCODING:
      first_layer_size += SHAPE_EMBEDDING_SIZE
    if USE_TYPE_ENCODING:
      first_layer_size += 2
    self.hid1 = T.nn.Linear(4*(first_layer_size+1), 500)
    self.hid2 = torch.nn.Linear(500, 250)
    self.hid3 = torch.nn.Linear(250, 100)
    self.oupt = torch.nn.Linear(100, len(api2indx))

    torch.nn.init.xavier_uniform_(self.hid1.weight)
    torch.nn.init.zeros_(self.hid1.bias)
    torch.nn.init.xavier_uniform_(self.hid2.weight)
    torch.nn.init.zeros_(self.hid2.bias)
    torch.nn.init.xavier_uniform_(self.oupt.weight)
    torch.nn.init.zeros_(self.oupt.bias)

    torch.nn.Dropout(p=0.2)

  def forward(self, x):
    z1 = torch.tanh(self.hid1(x))
    z2 = torch.tanh(self.hid2(z1))
    z3 = torch.tanh(self.hid3(z2))
    z = self.oupt(z3)  # no softmax: CrossEntropyLoss()
    return (z, z3, z2, z1)


In [ ]:
model = Net(len(api2indx)).to(device)
checkpoint = torch.load(EMBEDDING_DATA_NAME + "_model.pt")
model.load_state_dict(checkpoint)
model.eval()

In [ ]:
def embed_tensors(input_list, output):
    ''' Embed the tensors '''
    it_pad = []

    for input_tensor in input_list:
        if input_tensor is None:
            encoding_size = EMBEDDING_SIZE
            if USE_SHAPE_ENCODING:
                encoding_size += SHAPE_EMBEDDING_SIZE
            if USE_TYPE_ENCODING:
                encoding_size += 2
            t = torch.zeros(encoding_size + 1)
            t[-1] = -1
            it_pad.append(t)
        else:
            it_pad.append(tensor_flatten_pad(input_tensor))
            
    for i in range(len(it_pad),3):
        encoding_size = EMBEDDING_SIZE
        if USE_SHAPE_ENCODING:
            encoding_size += SHAPE_EMBEDDING_SIZE
        if USE_TYPE_ENCODING:
            encoding_size += 2
        t = torch.zeros(encoding_size + 1)
        t[-1] = -1
        it_pad.append(t)

    ot_pad = tensor_flatten_pad(output)
    domain_embedding = T.flatten(T.stack((it_pad[0], it_pad[1], it_pad[2], ot_pad)))
    return(domain_embedding)


In [ ]:
import time
import random

In [ ]:
num_benchmarks = 0
top_1_correct = 0
top_3_correct = 0
top3_logs = []

In [ ]:
def predict_api(domain_embedding, ground_truth, benchmark_id, threshold=0, top_n=3, print_log=True):
    '''
    Call the model
    '''
    global num_benchmarks, top_1_correct, top_3_correct
    with T.no_grad():
        start_time = time.time()
        predicts, _, _, _ = model(domain_embedding)
        ms =  (time.time() - start_time)*1000
        if print_log:
            print("--- %s micro seconds ---" % (ms))

    softmax_list = torch.nn.Softmax()
    confidence = softmax_list(predicts)
    num_gt_threshold = sum(c > threshold for c in confidence)

    predicted_api_list = (torch.argsort(predicts, descending=True)).numpy()
    topn_list = predicted_api_list[:min(top_n, num_gt_threshold)]

    '''
    Predict API functions
    '''
    predicted_api_list = (T.argsort(predicts)).numpy()
    if print_log:
        print('---- Expected: %s' % ground_truth)

    topn_operations = [list(api2indx.keys())[list(api2indx.values()).index(api)] for api in topn_list]
    topn_confidences = [confidence[api].item() for api in topn_list]
    
    result_str = ", ".join(["{} ({:.2f})".format(op, c) for op, c in zip(topn_operations, topn_confidences)])
    if print_log:
        print("Predicted operations: {}".format(result_str))
    top3_logs.append("{:8s}: {}".format(benchmark_id, result_str))
    
    num_benchmarks += 1
    if len(topn_operations) > 0:
        if isinstance(ground_truth, list):
            if topn_operations[0] in ground_truth:
                top_1_correct += 1
            if len(list(set(ground_truth) & set(topn_operations[:3]))) > 0:
                top_3_correct += 1
        else:
            if ground_truth == topn_operations[0]:
                top_1_correct += 1
            if ground_truth in topn_operations[:3]:
                top_3_correct += 1

In [ ]:
# Run this cell after running cells below to calculate top-k accuracy
print("Number of benchmarks: {}".format(num_benchmarks))
print("Top-1 accuracy: {}".format(top_1_correct / num_benchmarks))
print("Top-3 accuracy: {}".format(top_3_correct / num_benchmarks))

In [ ]:
# Run this cell to print top-3 prediction of the benchmarks
print("\n".join(top3_logs))

# Stack Overflow Benchmarks
Below cells contain Stack Overflow benchmarks input and output. 
To compute the top-k accuracy, run all cells below, and run the above two cells.

In [ ]:
def Iterative01_01():
  in1 = torch.tensor([[5, 2], [1, 3], [0, 2]])
  out = torch.tensor([[[ 5,  2],
         [ 1,  3],
         [ 0, 2]],
        [[ 5,  2],
         [ 1,  3],
         [ 0, 2]]])

  print(in1, in1.shape)
  print(out, out.shape)
  print(in1.expand((2, 3, 2)))

  domain_embedding = embed_tensors([in1], out)
  
  predict_api(domain_embedding, 'expand', 'SO01-01')

Iterative01_01()

In [ ]:
def Iterative01_02():
    in1 = torch.tensor([[5, 2], [1, 3], [0, 2]])
    in1 = in1.expand((2, 3, 2))
    out = torch.tensor([[[5, 5], [1, 1], [0, 0]],
                                [[2, 2], [3, 3], [2, 2]]])
                                
    print(in1, in1.shape)
    print(out, out.shape)
    print(torch.transpose(in1, 0, 2))

    domain_embedding = embed_tensors([in1], out)
    print(domain_embedding.shape)

    predict_api(domain_embedding, 'transpose', 'SO01-02')

Iterative01_02()

In [ ]:
def Iterative02_01():
  in1 = torch.tensor([5, 1, 0, 3, 0, 0, 2, 0, 2])
  in2 = 1
  out = torch.lt(in1, in2)

  print(in1, in1.shape)
  print(in2)
  print(out, out.shape)
  print(torch.lt(in1, in2))

  domain_embedding = embed_tensors([in1, in2], out)
  predict_api(domain_embedding, 'lt', 'SO02-01')
  
Iterative02_01()

In [ ]:
def Iterative02_02():
  in1 = torch.tensor([False, False,  True, False,  True,  True, False,  True, False])
  in2 = torch.tensor([5, 1, 0, 3, 0, -1, 2, -10, 2])
  in3 = 1
  out = torch.tensor([1, 1, 0, 1, 0, -1, 1, -10, 1])

  print(in1, in1.shape)
  print(in2, in2.shape)
  print(in3)
  print(out, out.shape)
  print(torch.where(in1, in2, in3))

  domain_embedding = embed_tensors([in1, in2, in3], out)

  predict_api(domain_embedding, 'where', 'SO02-02')

Iterative02_02()

In [ ]:
def Iterative05():
    in1 = torch.tensor([[4, 3, 1], [6, 5, 2]])
    in2 = torch.tensor([[[5, 5]], [[1, 5]], [[6, 0]]])
    out = torch.tensor([[[29, 35]], [[47, 55]]])

    print(in1, in1.shape, in1.dtype)
    print(in2, in2.shape, in2.dtype)
    print(out, out.shape, out.dtype)
    print(torch.tensordot(in1, in2, 1))

    domain_embedding = embed_tensors([in1, in2], out)

    predict_api(domain_embedding, 'tensordot', 'SO05')

Iterative05()

In [ ]:
def Iterative06_01():
    in1 = torch.tensor([3, 5, 0, 2, 3, 3, 0])
    out = torch.tensor([[3],
        [5],
        [0],
        [2],
        [3],
        [3],
        [0]])

    print(in1, in1.shape, in1.dtype)
    print(out, out.shape, out.dtype)
    print(torch.unsqueeze(in1, 1))

    domain_embedding = embed_tensors([in1], out)
    predict_api(domain_embedding, ['unsqueeze', 'reshape'], 'SO06-01')

Iterative06_01()

In [ ]:
def Iterative06_02():
    in1 = torch.tensor([3, 5, 0, 2, 3, 3, 0])
    in2 = torch.unsqueeze(in1, 1)
    out = torch.tensor([
                [1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0],
                [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0],
                [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0],
                [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0],
                [1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0],
                [1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0],
                [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0],
            ]).bool()

    print(in1, in1.shape, in1.dtype)
    print(in2, in2.shape, in2.dtype)
    print(out, out.shape, out.dtype)
    print(torch.eq(in1, in2).float())

    domain_embedding = embed_tensors([in1, in2], out)
    predict_api(domain_embedding, 'eq', 'SO06-02')

Iterative06_02()

In [ ]:
def Iterative07():
    in1 = torch.tensor([
                    [[8, 4, 6], [2, 12, 3]],
                    [[11, 12, 5], [9, 12, 12]],
                    [[9, 2, 13], [7, 0, 7]],
                    [[2, 10, 5], [7, 1, 2]],
                ])
    out = torch.tensor([
                [[8, 4, 6], [11, 12, 5], [9, 2, 13], [2, 10, 5]],
                [[2, 12, 3], [9, 12, 12], [7, 0, 7], [7, 1, 2]],
            ])

    print(in1, in1.shape, in1.dtype)
    print(out, out.shape, out.dtype)
    print(torch.transpose(in1, 0, 1))

    domain_embedding = embed_tensors([in1], out)

    predict_api(domain_embedding, 'transpose', 'SO07')

Iterative07()

In [ ]:
def Iterative08_01():
    in1 = torch.tensor([1, 0, 0, 2, 1, 3, 5, 0, 1, 2, 10])
    in2 = 1
    out = torch.tensor([False, False, False,  True, False,  True,  True, False, False,  True, True])

    print(in1, in1.shape, in1.dtype)
    print(in2)
    print(out, out.shape, out.dtype)
    print(torch.gt(in1, in2))

    domain_embedding = embed_tensors([in1, in2], out)

    predict_api(domain_embedding, 'gt', 'SO08-01')

Iterative08_01()

In [ ]:
def Iterative08_02():
    in1 = torch.tensor([12, 3, 45, 6, 7, 8, 9, 87, 65, 4, 32])
    in2 = torch.tensor([1, 0, 0, 2, 1, 3, 5, 0, 1, 2, 10])
    in2 = torch.gt(in2, 1)
    out = torch.tensor([6, 8, 9, 4, 32])

    print(in1, in1.shape, in1.dtype)
    print(in2, in2.shape, in2.dtype)
    print(out, out.shape, out.dtype)
    print(torch.masked_select(in1, in2))

    domain_embedding = embed_tensors([in1, in2], out)

    predict_api(domain_embedding, 'masked_select', 'SO08-02')

Iterative08_02()

In [ ]:
def Iterative11():
    in1 = torch.tensor([4, 0, 1, 1, 0, 4, 0, 0, 3, 4, 1])
    out = torch.tensor([4, 3, 0, 1, 3])

    print(in1, in1.shape, in1.dtype)
    print(out, out.shape, out.dtype)
    print(torch.bincount(in1))

    domain_embedding = embed_tensors([in1], out)

    predict_api(domain_embedding, 'bincount', 'SO11')

Iterative11()

In [ ]:
def Iterative13_01():
    in1 = torch.tensor([[3, 5], [10, 2]])
    in2 = torch.tensor([[[1, 0], [5, 4]], [[3, 10], [2, 0]]])
    out = torch.tensor([[[28, 20], [19, 30]], [[20, 8], [34, 100]]])

    print(in1, in1.shape, in1.dtype)
    print(in2, in2.shape, in2.dtype)
    print(out, out.shape, out.dtype)
    print(torch.matmul(in1, in2))

    domain_embedding = embed_tensors([in1, in2], out)
    predict_api(domain_embedding, 'matmul', 'SO13-01')

Iterative13_01()

In [ ]:
def Iterative13_02():
    in1 = torch.tensor([[3, 5], [10, 2]])
    in2 = torch.tensor([[[1, 0], [5, 4]], [[3, 10], [2, 0]]])
    in1 = torch.matmul(in1, in2)
    out = torch.tensor([[[28, 20], [19, 30]], [[20, 8], [34, 100]]])

    print(in1, in1.shape, in1.dtype)
    print(out, out.shape, out.dtype)
    print(torch.transpose(in1, 0, 1))

    domain_embedding = embed_tensors([in1], out)
    predict_api(domain_embedding, 'transpose', "SO13-02")

Iterative13_02()

In [ ]:
def Iterative14():
    in1 = torch.tensor([
                    [
                        [False, False, True],
                        [False, False, False],
                        [True, False, True],
                        [False, True, False],
                        [False, False, False],
                        [True, True, True],
                        [True, True, False],
                    ]
                ])
    out = torch.tensor([[True, False, True, True, False, True, True]])

    print(in1, in1.shape, in1.dtype)
    print(out, out.shape, out.dtype)
    print(torch.any(in1, -1))

    domain_embedding = embed_tensors([in1], out)
    predict_api(domain_embedding, ['any', 'sum'], "SO14")

Iterative14()

In [ ]:
def Iterative15_01():
    in1 = torch.tensor([3, 1, 2, 0, 1, 0, 10, 1, 0])
    out = torch.tensor([ True, False,  True,  True, False,  True,  True, False,  True])

    print(in1, in1.shape, in1.dtype)
    print(out, out.shape, out.dtype)
    print(torch.ne(in1, 1))

    domain_embedding = embed_tensors([in1], out)

    predict_api(domain_embedding, 'ne', "SO15-01")

Iterative15_01()

In [ ]:
def Iterative15_02():
    in1 = torch.tensor([3, 1, 2, 0, 1, 0, 10, 1, 0])
    in2 = torch.ne(in1, 1)
    out = torch.tensor([3, 0, 2, 0, 0, 0, 10, 0, 0])

    print(in1, in1.shape, in1.dtype)
    print(in2, in2.shape, in2.dtype)
    print(out, out.shape, out.dtype)
    print(torch.mul(in1, in2))

    domain_embedding = embed_tensors([in1, in2], out)

    predict_api(domain_embedding, 'mul', 'SO15-02')
    print(torch.div(in1, in2))

Iterative15_02()

In [ ]:
def Iterative16_01():
    in1 = torch.tensor([4, 10, 6])
    out = torch.tensor([[ 4],
        [10],
        [6]])

    print(in1, in1.shape, in1.dtype)
    print(out, out.shape, out.dtype)
    print(torch.unsqueeze(in1, 1))

    domain_embedding = embed_tensors([in1], out)

    predict_api(domain_embedding, ['unsqueeze', 'reshape'], 'SO16-01')

Iterative16_01()

In [ ]:
def Iterative16_02():
    in1 = torch.tensor([[2, 5], [3, 0], [8, 7]])
    in2 = torch.tensor([4, 10, 6])
    in2 = torch.unsqueeze(in2, 1)
    out = torch.tensor([[8, 20], [30, 0], [48, 42]])

    print(in1, in1.shape, in1.dtype)
    print(in2, in2.shape, in2.dtype)
    print(out, out.shape, out.dtype)
    print(torch.mul(in1, in2))

    domain_embedding = embed_tensors([in1, in2], out)

    predict_api(domain_embedding, 'mul', 'SO16-02')

Iterative16_02()

In [ ]:
def Iterative17():
    in1 = torch.tensor([17, 32, 99])
    in2 = in1
    out = torch.tensor([[17, 17], [32, 32], [99, 99]])

    print(in1, in1.shape, in1.dtype)
    print(in2, in2.shape, in2.dtype)
    print(out, out.shape, out.dtype)
    print(torch.stack((in1, in2), 1))

    domain_embedding = embed_tensors([in1, in2], out)

    predict_api(domain_embedding, 'stack', 'SO17')

Iterative17()

In [ ]:
def Iterative18_01():
    in1 = torch.tensor([[[1, 1, 1], [1, 0, 1]], [[1, 2, 3], [4, 5, 6]]])
    in2 = torch.tensor([[1, 1, 1, 1], [1, 2, 3, 4], [5, 6, 7, 8]])
    out = torch.tensor([[[ 7,  9, 11, 13],
         [ 6,  7,  8,  9]],

        [[18, 23, 28, 33],
         [39, 50, 61, 72]]])

    print(in1, in1.shape, in1.dtype)
    print(in2, in2.shape, in2.dtype)
    print(out, out.shape, out.dtype)
    print(torch.matmul(in1, in2))

    domain_embedding = embed_tensors([in1, in2], out)

    predict_api(domain_embedding, 'matmul', 'SO18-01')

Iterative18_01()

In [ ]:
def Iterative18_02():
    in1 = torch.tensor([100, 200, 300, 400])
    in2 = torch.matmul(torch.tensor([[[1, 1, 1], [1, 0, 1]], [[1, 2, 3], [4, 5, 6]]]), torch.tensor([[1, 1, 1, 1], [1, 2, 3, 4], [5, 6, 7, 8]]))
    out = torch.tensor([
                [[107, 209, 311, 413], [106, 207, 308, 409]],
                [[118, 223, 328, 433], [139, 250, 361, 472]],
            ])

    print(in1, in1.shape, in1.dtype)
    print(in2, in2.shape, in2.dtype)
    print(out, out.shape, out.dtype)
    print(torch.add(in1, in2))

    domain_embedding = embed_tensors([in2, in1], out)

    predict_api(domain_embedding, 'add', 'SO18-02')

Iterative18_02()

In [ ]:
def Iterative20_01():
    in1 = torch.tensor([
                    [7, 2, 1],
                    [4, 5, 1],
                    [4, 4, 2],
                    [3, 4, 3],
                    [0, 0, 1],
                ])
    out = torch.tensor([0, 1, 0, 1, 2])

    print(in1, in1.shape, in1.dtype)
    print(out, out.shape, out.dtype)
    print(torch.argmax(in1, 1))

    domain_embedding = embed_tensors([in1], out)
    predict_api(domain_embedding, 'argmax', 'SO20-01')

Iterative20_01()

In [ ]:
def Iterative20_02():
    in1 = torch.tensor([
                    [7, 2, 1],
                    [4, 5, 1],
                    [4, 4, 2],
                    [3, 4, 3],
                    [0, 0, 1],
                ])
    in1 = torch.argmax(in1, 1)
    out = torch.tensor([[1, 0, 0], [0, 1, 0], [1, 0, 0], [0, 1, 0], [0, 0, 1]])

    print(in1, in1.shape, in1.dtype)
    print(out, out.shape, out.dtype)
    print(torch.nn.functional.one_hot(in1, 3))

    domain_embedding = embed_tensors([in1], out)
    predict_api(domain_embedding, 'one_hot', 'SO20-02')

Iterative20_02()

In [ ]:
def Iterative21():
    in1 = torch.tensor([[2, 5, 3], [1, 3, 6], [1, 6, 3], [7, 0, 3]])
    in2 = torch.tensor([[2], [0], [1], [0]])
    out = torch.tensor([[3], [1], [6], [7]])

    print(in1, in1.shape, in1.dtype)
    print(in2, in2.shape, in2.dtype)
    print(out, out.shape, out.dtype)
    print(torch.gather(in1, 1, in2))

    domain_embedding = embed_tensors([in1, in2], out)

    predict_api(domain_embedding, 'gather', "SO21")

Iterative21()

In [ ]:
def Iterative22():
    in1 = torch.tensor([3, 1, 10])
    in2 = torch.tensor([[6, 4], [5, 1], [3, 4]])
    out = torch.tensor([53, 53])

    print(in1, in1.shape, in1.dtype)
    print(in2, in2.shape, in2.dtype)
    print(out, out.shape, out.dtype)
    print(torch.tensordot(in1, in2, 1))

    domain_embedding = embed_tensors([in1, in2], out)

    predict_api(domain_embedding, ['tensordot', 'matmul'], 'SO22')
    print(torch.matmul(in1, in2))

Iterative22()

In [ ]:
def Iterative23_01():
    in1 = torch.tensor([[0, 5, 2], [3, 1, 4], [5, 1, 5]])
    out = torch.tensor([[[1, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 1, 0, 0, 0],
         [0, 0, 1, 0, 0, 0, 0, 0, 0]],

        [[0, 0, 0, 1, 0, 0, 0, 0, 0],
         [0, 1, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 1, 0, 0, 0, 0]],

        [[0, 0, 0, 0, 0, 1, 0, 0, 0],
         [0, 1, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 1, 0, 0, 0]]])

    print(in1, in1.shape, in1.dtype)
    print(out, out.shape, out.dtype)
    print(torch.nn.functional.one_hot(in1, 9))

    domain_embedding = embed_tensors([in1], out)
    predict_api(domain_embedding, 'one_hot', 'SO23-01')
    print(torch.sum(in1, 1))

Iterative23_01()

In [ ]:
def Iterative23_02():
    in1 = torch.tensor([[0, 5, 2], [3, 1, 4], [5, 1, 5]])
    in1 = torch.nn.functional.one_hot(in1, 9)
    out = torch.tensor([
                [1, 0, 1, 0, 0, 1, 0, 0, 0],
                [0, 1, 0, 1, 1, 0, 0, 0, 0],
                [0, 1, 0, 0, 0, 1, 0, 0, 0],
            ])

    print(in1, in1.shape, in1.dtype)
    print(out, out.shape, out.dtype)
    print(torch.max(in1, 1)[0])

    domain_embedding = embed_tensors([in1], out)
    predict_api(domain_embedding, ['max', 'sum'], 'SO23-02')

Iterative23_02()

In [ ]:
def Iterative24_01():
  in1 = torch.tensor([1, 0, 2, 0, 1, 1, 0, 2])
  in2 = 0
  out = torch.tensor([ True, False,  True, False,  True,  True, False,  True])

  print(in1, in1.shape)
  print(in2)
  print(out, out.shape)
  print(torch.ne(in1, in2))

  domain_embedding = embed_tensors([in1, in2], out)

  predict_api(domain_embedding, 'ne', 'SO24-01')

Iterative24_01()

In [ ]:
def Iterative24_02():
  in1 = torch.tensor([3, 1, 4, 5, 2, 8, 6, 7])
  in2 = torch.tensor([1, 0, 2, 0, 1, 1, 0, 2])
  out = torch.divide(in1, in2)

  print(in1, in1.shape)
  print(in2, in2.shape)
  print(out, out.shape)
  print(torch.divide(in1, in2))

  domain_embedding = embed_tensors([in1, in2], out)

  predict_api(domain_embedding, 'divide', 'SO24-02')

Iterative24_02()

In [ ]:
def Iterative24_03():
  in1 = torch.tensor([ True, False,  True, False,  True,  True, False,  True])
  in4 = torch.tensor([1, 0, 2, 0, 1, 1, 0, 2])
  in3 = torch.tensor([3, 1, 4, 5, 2, 8, 6, 7]).float()
  in2 = torch.divide(in3, in4)
  out = torch.tensor([3, 1, 2, 5, 2, 8, 6, 3.5])

  print(in1, in1.shape)
  print(in2, in2.shape)
  print(in3, in3.shape)
  print(out, out.shape)
  print(torch.where(in1, in2, in3))

  domain_embedding = embed_tensors([in1, in2, in3], out)
  predict_api(domain_embedding, 'where', 'SO24-03')

Iterative24_03()

In [ ]:
def Iterative25_01():
    in1 = 3
    out = torch.tensor([[1, 0, 0],
                        [0, 1, 0],
                        [0, 0, 1]])

    print(in1)
    print(out, out.shape, out.dtype)
    print(torch.eye(in1))

    domain_embedding = embed_tensors([in1], out)

    predict_api(domain_embedding, 'eye', 'SO25-01')

Iterative25_01()

In [ ]:
def Iterative25_02():
    in1 = torch.tensor([[1, 0, 0],
                        [0, 1, 0],
                        [0, 0, 1]])
    out = torch.tensor([
                [1, 0, 0],
                [0, 1, 0],
                [0, 0, 1],
                [1, 0, 0],
                [0, 1, 0],
                [0, 0, 1],
                [1, 0, 0],
                [0, 1, 0],
                [0, 0, 1],
                [1, 0, 0],
                [0, 1, 0],
                [0, 0, 1],
            ]).int()

    print(in1)
    print(out, out.shape, out.dtype)
    print(torch.tile(in1, (4, 1)))

    domain_embedding = embed_tensors([in1], out)

    predict_api(domain_embedding, 'tile', 'SO25-02')

Iterative25_02()

In [ ]:
def Iterative26_01():
    in1 = torch.tensor([[[3, 4], [1, 2]], [[5, 2], [10, 3]], [[10, 20], [4, 7]]])
    out = torch.tensor([[ 4,  6],
                        [15,  5],
                        [ 14, 27]])

    print(in1, in1.shape, in1.dtype)
    print(out, out.shape, out.dtype)
    print(torch.sum(in1, 1))

    domain_embedding = embed_tensors([in1], out)

    predict_api(domain_embedding, 'sum', 'SO26-01')

Iterative26_01()

In [ ]:
def Iterative26_02():
    in1 = torch.tensor([[[3, 4], [1, 2]], [[5, 2], [10, 3]], [[10, 20], [4, 7]]])
    in1 = torch.sum(in1, 1)
    out = torch.tensor([10, 20, 41])

    print(in1, in1.shape, in1.dtype)
    print(out, out.shape, out.dtype)
    print(torch.sum(in1, 1))

    domain_embedding = embed_tensors([in1], out)

    predict_api(domain_embedding, 'sum', 'SO26-02')

Iterative26_02()

In [ ]:
def Iterative27_01():
    in1 = torch.tensor([0, 3, 5, 6])
    out = torch.tensor([[1, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 1, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 0, 0, 1, 0]])

    print(in1, in1.shape, in1.dtype)
    print(out, out.shape, out.dtype)
    print(torch.nn.functional.one_hot(in1, 8))

    domain_embedding = embed_tensors([in1], out)

    predict_api(domain_embedding, 'one_hot', 'SO27-01')

Iterative27_01()

In [ ]:
def Iterative27_02():
    in1 = torch.tensor([0, 3, 5, 6])
    in1 = torch.nn.functional.one_hot(in1, 8)
    out = torch.tensor([1, 0, 0, 1, 0, 1, 1, 0])

    print(in1, in1.shape, in1.dtype)
    print(out, out.shape, out.dtype)
    print(torch.sum(in1, 0))

    domain_embedding = embed_tensors([in1], out)

    predict_api(domain_embedding, ['sum', 'any', 'max'], 'SO27-02')

Iterative27_02()

In [ ]:
def Iterative29():
    in1 = torch.tensor([1, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21])
    in2 = torch.tensor([12, 0, 10, 23, 16])
    out = torch.tensor([6, 0, 5, 11, 8])

    print(in1, in1.shape, in1.dtype)
    print(in2, in2.shape, in2.dtype)
    print(out, out.shape, out.dtype)
    print(torch.searchsorted(in1, in2))

    domain_embedding = embed_tensors([in1, in2], out)

    predict_api(domain_embedding, 'searchsorted', 'SO29')

Iterative29()

In [ ]:
import math
def Iterative30():
    in1 = torch.tensor([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])
    in2 = torch.tensor([[9.0, 4.0], [8.0, 5.0], [7.0, 6.0]])
    out = torch.tensor([
                [math.sqrt(68), math.sqrt(58), math.sqrt(52)],
                [math.sqrt(36), math.sqrt(26), math.sqrt(20)],
                [math.sqrt(20), math.sqrt(10), math.sqrt(4)],
            ])

    print(in1, in1.shape, in1.dtype)
    print(in2, in2.shape, in2.dtype)
    print(out, out.shape, out.dtype)
    print(torch.cdist(in1, in2))

    domain_embedding = embed_tensors([in1, in2], out)

    predict_api(domain_embedding, 'cdist', 'SO30')

Iterative30()

In [ ]:
def Iterative32_01():
    in1 = 4
    out = torch.tensor([0, 1, 2, 3])

    print(in1)
    print(out, out.shape, out.dtype)
    print(torch.arange(in1))

    domain_embedding = embed_tensors([in1], out)

    predict_api(domain_embedding, 'arange', 'SO32-01')

Iterative32_01()

In [ ]:
def Iterative32_02():
    in1 = torch.tensor([[1, 6, 2, 1], [3, 1, 4, 2], [2, 1, 2, 5]])
    in2 = torch.arange(4)
    out = torch.tensor([13, 15, 20])

    print(in1, in1.shape, in1.dtype)
    print(in2, in2.shape, in2.dtype)
    print(out, out.shape, out.dtype)
    print(torch.tensordot(in1, in2, 1))

    domain_embedding = embed_tensors([in1, in2], out)

    predict_api(domain_embedding, ['tensordot', 'matmul'], 'SO32-02')

Iterative32_02()

In [ ]:
def Iterative34():
    in1 = torch.tensor([3, 5, 10])
    in2 = torch.tensor([[[1, 2], [3, 4]], [[5, 6], [7, 8]], [[10, 20], [30, 40]]])
    out = torch.tensor([[128, 236], [344, 452]])

    print(in1, in1.shape, in1.dtype)
    print(in2, in2.shape, in2.dtype)
    print(out, out.shape, out.dtype)
    print(torch.tensordot(in1, in2, 1))

    domain_embedding = embed_tensors([in1, in2], out)

    predict_api(domain_embedding, 'tensordot', 'SO34')

Iterative34()

In [ ]:
def Iterative36_01():
    in1 = 8
    out = torch.tensor([0, 1, 2, 3, 4, 5, 6, 7])

    print(in1)
    print(out, out.shape, out.dtype)
    print(torch.arange(in1))

    domain_embedding = embed_tensors([in1], out)

    predict_api(domain_embedding, 'arange', 'SO36-01')

Iterative36_01()

In [ ]:
def Iterative36_02():
    in1 = torch.tensor([1, 0, 1, 1, 0, 1, 0, 1])
    in2 = torch.arange(8)
    out = torch.tensor([1, 1, 3, 4, 4, 6, 6, 8])

    print(in1, in1.shape, in1.dtype)
    print(in2, in2.shape, in2.dtype)
    print(out, out.shape, out.dtype)
    print(torch.add(in1, in2))

    domain_embedding = embed_tensors([in1, in2], out)

    predict_api(domain_embedding, 'add', 'SO36-02')

Iterative36_02()

In [ ]:
def Iterative36_03():
    in1 = torch.tensor([1, 0, 1, 1, 0, 1, 0, 1])
    in2 = torch.add(in1, torch.arange(8))
    out = torch.tensor([1.0, 0.0, 0.333333, 0.25, 0.0, 0.166667, 0.0, 0.125])

    print(in1, in1.shape, in1.dtype)
    print(in2, in2.shape, in2.dtype)
    print(out, out.shape, out.dtype)
    print(torch.div(in1, in2))

    domain_embedding = embed_tensors([in1, in2], out)

    predict_api(domain_embedding, 'div', 'SO36-03')

Iterative36_03()

In [ ]:
def Iterative37():
    in1 = torch.tensor([
                    [
                        [[10, 20, 30], [40, 50, 60]],
                        [[12, 34, 56], [78, 98, 76]],
                    ]
                ])
    in2 = torch.tensor([5, 10, 20])
    out = torch.tensor([[[850, 1900], [1520, 2890]]])

    print(in1, in1.shape, in1.dtype)
    print(in2, in2.shape, in2.dtype)
    print(out, out.shape, out.dtype)
    print(torch.tensordot(in1, in2, 1))

    domain_embedding = embed_tensors([in1, in2], out)

    predict_api(domain_embedding, ['tensordot', 'matmul'], 'SO37')
    print(torch.matmul(in1, in2))

Iterative37()

In [ ]:
def Iterative39_01():
    in1 = torch.tensor([[15, 10, 9, 20], [11, 0, 1, 9], [10, 1, 11, 25]])
    in2 = torch.tensor([[ True,  True, False,  True],
        [ True, False, False, False],
        [ True, False,  True,  True]])
    out = torch.tensor([[15, 10, 0, 20], [11, 0, 0, 0], [10, 0, 11, 25]])

    print(in1, in1.shape, in1.dtype)
    print(in2, in2.shape, in2.dtype)
    print(out, out.shape, out.dtype)
    print(torch.mul(in1, in2))

    domain_embedding = embed_tensors([in1, in2], out)

    predict_api(domain_embedding, 'mul', 'SO39-01')

Iterative39_01()

In [ ]:
def Iterative39_02():
    in1 = torch.tensor([[15, 10, 9, 20], [11, 0, 1, 9], [10, 1, 11, 25]])
    in1 = torch.mul(in1, in1.long().bool())
    out = torch.tensor([
                [225, 100, 81, 400],
                [121, 0, 1, 81],
                [100, 1, 121, 625],
            ])

    print(in1, in1.shape, in1.dtype)
    print(out, out.shape, out.dtype)
    print(torch.square(in1))

    domain_embedding = embed_tensors([in1], out)

    predict_api(domain_embedding, 'square', 'SO39-02')

Iterative39_02()

In [ ]:
def Iterative41_01():
    in1 = 10
    out = torch.tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

    print(in1)
    print(out, out.shape, out.dtype)
    print(torch.arange(in1))

    domain_embedding = embed_tensors([in1], out)

    predict_api(domain_embedding, 'arange', 'SO41-01')

Iterative41_01()

In [ ]:
def Iterative41_02():
    in1 = torch.tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
    in2 = 3
    out = torch.tensor([ True,  True,  True, False,  True,  True,  True,  True,  True,  True])

    print(in1, in1.shape, in1.dtype)
    print(in2)
    print(out, out.shape, out.dtype)
    print(torch.ne(in1, in2))

    domain_embedding = embed_tensors([in1, in2], out)

    predict_api(domain_embedding, 'ne', 'SO41-02')

Iterative41_02()

In [ ]:
def Iterative41_03():
    in1 = torch.tensor([5, 2, 8, 2, 4, 1, 1, 0, 2, 1])
    in2 = torch.ne(torch.arange(10), 3)
    out = torch.tensor([5, 2, 8, 4, 1, 1, 0, 2, 1])

    print(in1, in1.shape, in1.dtype)
    print(in2, in2.shape, in2.dtype)
    print(out, out.shape, out.dtype)
    print(torch.masked_select(in1, in2))

    domain_embedding = embed_tensors([in1, in2], out)

    predict_api(domain_embedding, 'masked_select', 'SO41-03')

Iterative41_03()

In [ ]:
def Iterative42():
    in1 = torch.tensor([4, 6, 2, 6, 7, 3, 3])
    in2 = 7
    out = torch.tensor([0, 0, 0, 0, 1, 0, 0])

    print(in1, in1.shape, in1.dtype)
    print(in2)
    print(out, out.shape, out.dtype)
    print(torch.eq(in1, 7))

    domain_embedding = embed_tensors([in1, in2], out)

    predict_api(domain_embedding, 'eq', 'SO42')

Iterative42()

In [ ]:
def Iterative44_01():
    in1 = torch.tensor([
                    [3, 5, 2],
                    [6, 2, 3],
                    [8, 7, 1],
                    [0, 3, 5],
                    [4, 7, 3],
                    [2, 1, 6],
                    [10, 20, 30],
                    [4, 5, 6],
                ])
    out = torch.tensor([[[ 3,  5],
         [ 2,  6]],

        [[ 2,  3],
         [ 8,  7]],

        [[ 1,  0],
         [3,  5]],

        [[4,  7],
         [ 3,  2]],

        [[ 1,  6],
         [10, 20]],

        [[30,  4],
         [ 5,  6]]])

    print(in1, in1.shape, in1.dtype)
    print(out, out.shape, out.dtype)
    print(torch.reshape(in1, (-1, 2, 2)))

    domain_embedding = embed_tensors([in1], out)

    predict_api(domain_embedding, 'reshape', 'SO44-01')

Iterative44_01()

In [ ]:
def Iterative44_02():
    in1 = torch.tensor([
                    [3, 5, 2],
                    [6, 2, 3],
                    [8, 7, 1],
                    [0, 3, 5],
                    [4, 7, 3],
                    [2, 1, 6],
                    [10, 20, 30],
                    [4, 5, 6],
                ])   
    in1 = torch.reshape(in1, (-1, 2, 2))
    out = torch.tensor([[9, 7, 5], [8, 19, 6], [6, 8, 9], [14, 25, 36]])

    print(in1, in1.shape, in1.dtype)
    print(out, out.shape, out.dtype)
    print(torch.sum(in1, 1))

    domain_embedding = embed_tensors([in1], out)

    predict_api(domain_embedding, 'sum', 'SO44-02')

Iterative44_02()

In [ ]:
def Iterative45_01():
  in1 = torch.tensor([1, 0, 1, 0, 1])
  out = torch.tensor([[1],
        [0],
        [1],
        [0],
        [1]])

  print(in1, in1.shape)
  print(out, out.shape)
  print(torch.unsqueeze(in1, 1))

  domain_embedding = embed_tensors([in1], out)
  
  predict_api(domain_embedding, 'unsqueeze', 'SO45-01')

Iterative45_01()

In [ ]:
def Iterative45_02():
  in1 = torch.tensor([[[12, 34], [56, 78], [23, 54], [76, 78], [42, 24]]])
  out = torch.tensor([[[34, 12],
         [78, 56],
         [54, 23],
         [78, 76],
         [24, 42]]])

  print(in1, in1.shape)
  print(out, out.shape)
  print(torch.roll(in1, 1, -1))

  domain_embedding = embed_tensors([in1], out)

  predict_api(domain_embedding, 'roll', 'SO45-02')

Iterative45_02()

In [ ]:
def Iterative45_03():
  in1 = torch.tensor([[1],
        [0],
        [1],
        [0],
        [1]]).bool()
  in2 = torch.tensor([[[34, 12],
         [78, 56],
         [54, 23],
         [78, 76],
         [24, 42]]])
  in3 = torch.tensor([[[12, 34], [56, 78], [23, 54], [76, 78], [42, 24]]])
  out = torch.tensor([[[34, 12],
         [56, 78],
         [54, 23],
         [76, 78],
         [24, 42]]])

  print(in1, in1.shape)
  print(in2, in2.shape)
  print(in3, in3.shape)
  print(out, out.shape)
  print(torch.where(in1, in2, in3))

  domain_embedding = embed_tensors([in1, in2, in3], out)

  predict_api(domain_embedding, 'where', 'SO45-03')

Iterative45_03()

In [ ]:
def Iterative46_01():
    in1 = 3
    out = torch.tensor([0, 1, 2])

    print(in1)
    print(out, out.shape, out.dtype)
    print(torch.arange(in1))

    domain_embedding = embed_tensors([in1], out)

    predict_api(domain_embedding, 'arange', 'SO46-01')

Iterative46_01()

In [ ]:
def Iterative46_02():
    in1 = torch.arange(3)
    in2 = torch.tensor([3, 4, 1])
    out = torch.tensor([0, 0, 0, 1, 1, 1, 1, 2])

    print(in1, in1.shape, in1.dtype)
    print(in2, in2.shape, in2.dtype)
    print(out, out.shape, out.dtype)
    print(torch.repeat_interleave(in1, in2, 0))

    domain_embedding = embed_tensors([in1, in2], out)

    predict_api(domain_embedding, 'repeat_interleave', 'SO46-02')

Iterative46_02()

In [ ]:
def Iterative48_01():
    in1 = torch.tensor([38, 53, 89, 38, 32, 64])
    out = torch.tensor([[38],
        [53],
        [89],
        [38],
        [32],
        [64]])

    print(in1, in1.shape, in1.dtype)
    print(out, out.shape, out.dtype)
    print(torch.unsqueeze(in1, 1))

    domain_embedding = embed_tensors([in1], out)
    predict_api(domain_embedding, ['unsqueeze', 'reshape'], 'SO48-01')

Iterative48_01()

In [ ]:
def Iterative48_02():
    in1 = torch.tensor([32, 53, 45, 38, 29, 89, 64, 23])
    in2 = torch.tensor([[38],
        [53],
        [89],
        [38],
        [32],
        [64]])
    out = torch.tensor([[False, False, False,  True, False, False, False, False],
        [False,  True, False, False, False, False, False, False],
        [False, False, False, False, False,  True, False, False],
        [False, False, False,  True, False, False, False, False],
        [ True, False, False, False, False, False, False, False],
        [False, False, False, False, False, False,  True, False]])

    print(in1, in1.shape, in1.dtype)
    print(in2, in2.shape, in2.dtype)
    print(out, out.shape, out.dtype)
    print(torch.eq(in1, in2))

    domain_embedding = embed_tensors([in1, in2], out)
    predict_api(domain_embedding, 'eq', 'SO48-02')

Iterative48_02()

In [ ]:
def Iterative48_03():
    in1 = torch.tensor([32, 53, 45, 38, 29, 89, 64, 23])
    in2 = torch.tensor([38, 53, 89, 38, 32, 64])
    in1 = torch.eq(in1, torch.unsqueeze(in2, 1)).long()
    out = torch.tensor([3, 1, 5, 3, 0, 6])

    print(in1, in1.shape, in1.dtype)
    print(out, out.shape, out.dtype)
    print(torch.argmax(in1, 1))

    domain_embedding = embed_tensors([in1], out)

    predict_api(domain_embedding, 'argmax', 'SO48-03')

Iterative48_03()

In [ ]:
def Iterative49_01():
    in1 = torch.tensor([
                    [[[1, 2, 3], [4, 5, 6]]],
                    [[[8, 10, 0], [6, 4, 2]]],
                    [[[9, 8, 7], [1, 2, 3]]],
                ])
    out = torch.tensor([[[[1, 8, 9],
                          [4, 6, 1]]],
                        [[[2, 10, 8],
                          [5, 4, 2]]],
                        [[[3, 0, 7],
                          [6, 2, 3]]]])

    print(in1, in1.shape, in1.dtype)
    print(out, out.shape, out.dtype)
    print(torch.transpose(in1, 0, 3))

    domain_embedding = embed_tensors([in1], out)
    predict_api(domain_embedding, 'transpose', 'SO49-01')

Iterative49_01()

In [ ]:
def Iterative49_02():
    in1 = torch.tensor([2.0, 0.5, 1.0])
    in2 = torch.tensor([[[[0.1000, 0.8000, 0.9000],
          [0.4000, 0.6000, 0.1000]]],


        [[[0.2000, 1.0000, 0.8000],
          [0.5000, 0.4000, 0.2000]]],


        [[[0.3000, 0.0000, 0.7000],
          [0.6000, 0.2000, 0.3000]]]])
    out = torch.tensor([[[[1, 8, 9],
                          [4, 6, 1]]],
                        [[[2, 10, 8],
                          [5, 4, 2]]],
                        [[[3, 0, 7],
                          [6, 2, 3]]]])

    print(in1, in1.shape, in1.dtype)
    print(in2, in2.shape, in2.dtype)
    print(out, out.shape, out.dtype)
    print(torch.mul(in1, in2))

    domain_embedding = embed_tensors([in1, in2], out)

    predict_api(domain_embedding, 'mul', 'SO49-02')

Iterative49_02()

In [ ]:
def Iterative49_03():
    in1 = torch.tensor([
                    [[[1, 2, 3], [4, 5, 6]]],
                    [[[8, 10, 0], [6, 4, 2]]],
                    [[[9, 8, 7], [1, 2, 3]]],
                ])
    in2 = torch.tensor([20, 5, 10])
    in1 = torch.mul(in2, torch.transpose(in1, 0, 3))
    out = torch.tensor([
                [[[20, 40, 60], [80, 100, 120]]],
                [[[40, 50, 0], [30, 20, 10]]],
                [[[90, 80, 70], [10, 20, 30]]],
            ])

    print(in1, in1.shape, in1.dtype)
    print(out, out.shape, out.dtype)
    print(torch.transpose(in1, 0, 3))

    domain_embedding = embed_tensors([in1], out)

    predict_api(domain_embedding, 'transpose', 'SO49-03')

Iterative49_03()

In [ ]:
def Iterative50_01():
  in1 = torch.tensor(3)
  out = torch.tensor([3, 3, 3, 3, 3])

  print(in1, in1.shape)
  print(out, out.shape)
  print(in1.expand((5)))

  domain_embedding = embed_tensors([in1], out)

  predict_api(domain_embedding, 'expand', 'SO50-01')

Iterative50_01()

In [ ]:
def Iterative50_02():
    in1 = torch.tensor(3).expand((5,))
    out = torch.tensor([
                [0, 0, 0, 1, 0, 0],
                [0, 0, 0, 1, 0, 0],
                [0, 0, 0, 1, 0, 0],
                [0, 0, 0, 1, 0, 0],
                [0, 0, 0, 1, 0, 0],
            ])

    print(in1, in1.shape, in1.dtype)
    print(out, out.shape, out.dtype)
    print(torch.nn.functional.one_hot(in1, 6))

    domain_embedding = embed_tensors([in1], out)

    predict_api(domain_embedding, 'one_hot', 'SO50-02')

Iterative50_02()

In [ ]:
print("Number of benchmarks: {}".format(num_benchmarks))
print("Top-1 precision: {}".format(top_1_correct / num_benchmarks))
print("Top-3 precision: {}".format(top_3_correct / num_benchmarks))

In [ ]:
print("\n".join(top3_logs))